# Define Spatial Extent of Project

Metropolitan Council NAD83 -- UTM zone 15N -- epsg:26915

Source: https://gisdata.mn.gov/dataset/us-mn-state-metc-bdry-census2010counties-ctus

Metadata: https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/bdry_census2010counties_ctus/metadata/metadata.html


In [5]:
# Import libraries

import os # For working with Operating System
import shutil # For deleting folders

import urllib # For accessing websites
import zipfile # For extracting from Zipfiles

import geopandas as gpd # Spatial Data Manipulation


In [24]:
# Download from source

# Retrieve file (~ 5mb)

urllib.request.urlretrieve("https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/bdry_census2010counties_ctus/shp_bdry_census2010counties_ctus.zip",
                           "shp_bdry_census2010counties_ctus.zip") 
# Extract from Zip

with zipfile.ZipFile("shp_bdry_census2010counties_ctus.zip","r") as zip_ref:
    zip_ref.extractall()
    zip_ref.close()

In [25]:
# Read, Select, and Save Data

metro = gpd.read_file(r"Census2010CTUs.shp") # Read data to geodataframe

mpls = metro[metro['CTU_NAME']=='Minneapolis'] # Select Minneapolis' shape

mpls.to_file(r'mpls_boundary.shp') # Save file

In [26]:
# Compile Shapefiles into zip

cwd = os.getcwd() # Current Working Directory

mpls_boundary = zipfile.ZipFile(cwd + '\\mpls_boundary.zip', 'w', zipfile.ZIP_DEFLATED) # Initialize folder

for root, dirs, files in os.walk(cwd):
    for file in files:
        if 'mpls' in file and file != 'mpls_boundary.zip': # Locate Shapefiles
            mpls_boundary.write(file) # Write to Zip
            
mpls_boundary.close() # Close zip

In [27]:
# Remove all files besides zip and notebook

for root, dirs, files in os.walk(cwd): # Loop over files & folders
    print(dirs)
    for file in files:
        if 'mpls_boundary.zip' == file: # Keep zip
            pass
        elif file != 'DefineBoundary.ipynb': # Keep notebook
            os.remove(file) # Remove all others
            
    for folder in dirs:
        shutil.rmtree(cwd+ '\\' + folder) # Remove folders

['metadata']
